In [9]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [10]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),  # 어떤걸 적용할지
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

0it [00:00, ?it/s]

Extracting MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/MNIST\raw


0it [00:00, ?it/s]

Extracting MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw


0it [00:00, ?it/s]

Extracting MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/MNIST\raw


0it [00:00, ?it/s]

Extracting MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw
Processing...
Done!


C:\Users\82102\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [13]:
# dataset loader
# drop_last : 마지막 batch를 버릴지 말지
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [15]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
device = torch.device('cpu')

In [16]:
# input : MNIST data image of shape 28 * 28 = 784
# output : 0~9 개의 숫자 - 10개의 클래스

linear = torch.nn.Linear(784, 10, bias=True).to(device)

In [17]:
# Initialization
# layer의 weight를 normal distribution으로 초기화 진행
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-0.3877, -2.8464,  0.3753,  ...,  1.1273,  0.4002, -0.7475],
        [ 0.9861,  0.2705,  0.4751,  ...,  0.8990,  0.2413, -0.1923],
        [ 0.3377,  0.2586,  0.1821,  ..., -0.2632, -1.4041, -1.2519],
        ...,
        [-1.7024,  0.3576,  0.9008,  ...,  0.2891,  0.2086, -0.6763],
        [ 1.4576,  0.8870, -0.1375,  ...,  0.5837,  0.5502, -0.9657],
        [-2.2537, -0.7459, -0.3891,  ...,  0.1949, -0.2207, -0.6301]],
       requires_grad=True)

In [18]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)

In [19]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 5.364820004
Epoch: 0002 cost = 1.695096016
Epoch: 0003 cost = 1.092520952
Epoch: 0004 cost = 0.861804903
Epoch: 0005 cost = 0.736090481
Epoch: 0006 cost = 0.654336989
Epoch: 0007 cost = 0.594461143
Epoch: 0008 cost = 0.549812913
Epoch: 0009 cost = 0.514277458
Epoch: 0010 cost = 0.486178041
Epoch: 0011 cost = 0.463221341
Epoch: 0012 cost = 0.444095254
Epoch: 0013 cost = 0.427998245
Epoch: 0014 cost = 0.413669497
Epoch: 0015 cost = 0.401724517
Learning finished


In [20]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8910999894142151
Label:  3
Prediction:  3


C:\Users\82102\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\82102\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


### mnist_nn

In [21]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [22]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),  # 어떤걸 적용할지
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [23]:
# dataset loader
# drop_last : 마지막 batch를 버릴지 말지
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [24]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
device = torch.device('cpu')

In [25]:
# input : MNIST data image of shape 28 * 28 = 784
# output : 0~9 개의 숫자 - 10개의 클래스

# nn layers
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [26]:
# Initialization
# layer의 weight를 normal distribution으로 초기화 진행

torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.9150, -1.1908,  0.6533,  ..., -0.9276,  0.5930, -0.2682],
        [-0.5529,  1.7574, -1.6959,  ...,  0.4586,  0.1022, -0.7408],
        [-0.4657, -1.4958,  1.6482,  ...,  0.7493,  0.1285,  0.5262],
        ...,
        [-0.4057,  0.7634,  0.2929,  ...,  0.3239, -1.3294,  1.1021],
        [ 1.1341,  1.5654, -0.5336,  ..., -1.2249, -0.4854, -0.0867],
        [ 2.1049, -0.7947, -0.5613,  ..., -0.2881, -1.4658, -0.6110]],
       requires_grad=True)

In [27]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

# 여기서 마지막에 활성함수를 써주지 않은 이유는, 아래에서 CrossEntropyLoss가 Softmax를 이미 포함하고 있기 때문이다.

In [28]:
# define cost/loss & optimizer
# CrossEntropyLossd에는 Softmax 가 이미 내제되어있다.

criterion = torch.nn.CrossEntropyLoss().to(device)   
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [29]:

total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 186.264373779
Epoch: 0002 cost = 38.880268097
Epoch: 0003 cost = 23.748645782
Epoch: 0004 cost = 16.168968201
Epoch: 0005 cost = 11.761745453
Epoch: 0006 cost = 8.541217804
Epoch: 0007 cost = 6.355021000
Epoch: 0008 cost = 4.708252907
Epoch: 0009 cost = 3.475087166
Epoch: 0010 cost = 2.591293335
Epoch: 0011 cost = 1.983745694
Epoch: 0012 cost = 1.515189886
Epoch: 0013 cost = 1.128218889
Epoch: 0014 cost = 0.914033830
Epoch: 0015 cost = 0.772756457
Learning finished


In [30]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.946399986743927
Label:  3
Prediction:  5


#### 다층 레이어를 쌓고 relu를 사용하므로써 정확도가 올라간것을 확인할 수 있다.